In [21]:
import sys
from pathlib import Path
import os

sys.path.append(str(Path(os.getcwd()).resolve().parents[1]))

from src.utils import *
from src.mappings import *

In [22]:
df = read_dbf('../data/raw/peaks.DBF')

In [23]:
df.head()

,peakid,pkname,pkname2,location,heightm,heightf,himal,region,open,unlisted,...,peakmemo,pyear,pseason,pexpid,psmtdate,pcountry,psummiters,psmtnote,refermemo,photomemo
0,AMAD,Ama Dablam,Amai Dablang,Khumbu Himal,6814,22356,12,2,True,False,...,"Other map altitudes:\r\n 6814m - HMG-MT, HMG...",1961,1,AMAD61101,Mar 13,"New Zealand, USA, UK","Mike Gill, Wally Romanes, Barry Bishop, Michae...",None,None,W Face (High 126:5 May 1993)\r\nSE Face (High ...
1,AMPG,Amphu Gyabjen,Amphu Gyabien,Khumbu Himal (N of Ama Dablam),5630,18471,12,2,True,False,...,"Other map altitudes:\r\n 5630m - HMG-Finn, N...",1953,1,AMPG53101,Apr 11,UK,"John Hunt, Tom Bourdillon",None,None,None
2,ANN1,Annapurna I,None,Annapurna Himal,8091,26545,1,5,True,False,...,"Other map altitudes:\r\n 8091m - HMG-MT, HMG...",1950,1,ANN150101,Jun 03,France,"Maurice Herzog, Louis Lachenal",None,Dyhrenfurth history 1950-1977 (MM 58:44-47 Nov...,S Face (High 122:3 Jan 1993) (Beghin accident)...
3,ANN2,Annapurna II,None,Annapurna Himal,7937,26040,1,5,True,False,...,"Other map altitudes:\r\n 7937m - HMG-MT, HMG...",1960,1,ANN260101,May 17,"UK, Nepal","Richard Grant, Chris Bonington, Ang Nyima Sherpa",None,Dyhrenfurth history 1960-1976 (MM 51:36-37 Sep...,N Face (MM 51:36 Sep 1976)
4,ANN3,Annapurna III,None,Annapurna Himal,7555,24787,1,5,True,False,...,"Other map altitudes:\r\n 7555m - HMG-MT, HMG...",1961,1,ANN361101,May 06,India,"Mohan S. Kohli, Sonam Gyatso, Sonam Girmi",None,None,S Side (MM 125:11 Jan 1989)\r\nSW Face (MM 71:...


In [24]:
df.describe()

,heightm,heightf,himal,region,phost,pstatus,pseason
count,490.000000,490.000000,490.000000,490.000000,490.000000,490.000000,490.000000
mean,6645.828571,21803.914286,10.310204,3.767347,2.130612,1.763265,1.577551
std,568.992482,1866.778780,5.555603,2.144798,1.521247,0.425513,1.249583
min,5407.000000,17740.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,6235.000000,20456.000000,6.000000,2.000000,1.000000,2.000000,1.000000
50%,6554.500000,21504.000000,11.000000,3.000000,1.000000,2.000000,1.000000
75%,6892.000000,22611.750000,15.000000,6.000000,4.000000,2.000000,3.000000
max,8849.000000,29032.000000,20.000000,7.000000,6.000000,2.000000,4.000000


In [25]:
df.isna().sum()

peakid          0
pkname          0
pkname2       231
location        1
heightm         0
heightf         0
himal           0
region          0
open            0
unlisted        0
trekking        0
trekyear      459
restrict      203
phost           0
pstatus         0
peakmemo        5
pyear         122
pseason         0
pexpid        124
psmtdate      127
pcountry      122
psummiters      6
psmtnote      414
refermemo     471
photomemo     412
dtype: int64

### Peak Primary Key

In [26]:
# verify that the peakid column satisfies the requirements of a primary key
assert df.peakid.nunique() == df.shape[0]
assert df.peakid.isna().sum() == 0

### Entity Relationships
#### Peak/Location

In [27]:
print(df.groupby('peakid', as_index=False).location.nunique().location.min())
print(df.groupby('peakid', as_index=False).location.nunique().location.max())

0
1


In [28]:
print(df.groupby('location', as_index=False).peakid.nunique().peakid.min())
print(df.groupby('location', as_index=False).peakid.nunique().peakid.max())

1
6


> A peak is associated with one location, and a location can be associated with multiple peaks (on-to-many)

#### Peak/Himal

In [29]:
print(df.groupby('peakid', as_index=False).himal.nunique().himal.min())
print(df.groupby('peakid', as_index=False).himal.nunique().himal.max())

1
1


In [30]:
print(df.groupby('himal', as_index=False).peakid.nunique().peakid.min())
print(df.groupby('himal', as_index=False).peakid.nunique().peakid.max())

6
79


> A peak is associated with exactly one mountain, and a mountain has multiple peaks (one-to-many)

#### Location/Himal

In [31]:
print(df.groupby('location', as_index=False).himal.nunique().himal.min())
print(df.groupby('location', as_index=False).himal.nunique().himal.max())

1
1


In [32]:
print(df.groupby('himal', as_index=False).location.nunique().location.min())
print(df.groupby('himal', as_index=False).location.nunique().location.max())

6
64


> A location is associated with exactly one mountain, and a mountain has multiple locations (one-to-many)

> there is a hierarchical relationship of peak to location to mountain

#### Location/Region

In [33]:
print(df.groupby('location', as_index=False).region.nunique().region.min())
print(df.groupby('location', as_index=False).region.nunique().region.max())

1
1


In [34]:
print(df.groupby('region', as_index=False).location.nunique().location.min())
print(df.groupby('region', as_index=False).location.nunique().location.max())

35
113


> A location is associated with exactly one region, and a region has multiple locations (one-to-many)

> there is a hierarchical relationship of peak to location to region

#### Mountain/Region

In [35]:
print(df.groupby('region', as_index=False).himal.nunique().himal.min())
print(df.groupby('region', as_index=False).himal.nunique().himal.max())

2
5


In [36]:
print(df.groupby('himal', as_index=False).region.nunique().region.min())
print(df.groupby('himal', as_index=False).region.nunique().region.max())

1
2


> a region can contain multiple mountains, and a mountain can span multiple regions (many-to-many)

#### Region/Countries

In [37]:
df['host_country'] = apply_map(df.phost, host_map).str.split(';')
df = df.explode('host_country', ignore_index=True)

In [38]:
print(df.groupby('host_country', as_index=False).region.nunique().region.min())
print(df.groupby('host_country', as_index=False).region.nunique().region.max())

2
7


In [39]:
print(df.groupby('region', as_index=False).host_country.nunique().host_country.min())
print(df.groupby('region', as_index=False).host_country.nunique().host_country.max())

2
3


> There is a many-to-many relationship between regions and countries

#### Mountains/Countries

In [40]:
print(df.groupby('host_country', as_index=False).himal.nunique().himal.min())
print(df.groupby('host_country', as_index=False).himal.nunique().himal.max())

3
20


In [41]:
print(df.groupby('himal', as_index=False).host_country.nunique().host_country.min())
print(df.groupby('himal', as_index=False).host_country.nunique().host_country.max())

1
3


> There is a many-to-many relationship between regions and countries